# CT Accidents Analysis

In this project, we will analyze and classify a 
[dataset](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents) containing US accident data 
from 2016 to 2021. The dataset is over 1 GB large and contains over 2 million rows — that's a lot, 
so we decided to narrow down the dataset to just accidents in Connecticut.

We start with a principal component analysis of the dataset's features, then use Naive Bayes to 
classify an accident's description with its severity, and then use logistic regression to 
identify any other relationship between the features and severtiy.

This dataset gave us some problems when it came to interpreting the results of our analysis. We believe this is due to biases in the dataset. First, the dataset unevenly represented accidents of severity 2. Correlating features may have been easier to identify with more samples from the extremes of the severity scale.  Secondly, while we had a seemingly large sample size, there are a little over 6 million accidents in the United States every year. Hence our sample size of 3 million over the span of five years is actually relatively small. Due to the missing data it is difficult to determine if the trends found in our data  are representative of general truths about traffic accidents in general, or just the biases present in our subset of traffic accidents.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
import seaborn as sns
from bokeh.palettes import Spectral10, Category10
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from bokeh.palettes import Category10, Category20, Category20b, Category20c
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

import seaborn as sns
output_notebook()

Loading BokehJS ...

In [2]:
df=pd.read_csv("CT_Accidents.csv", low_memory=False)
df.head()

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,31581,A-31582,2,2016-11-30 15:58:59,2016-11-30 21:58:59,41.77461,-72.57781,41.76576,-72.62613,2.564,...,False,False,False,False,False,False,Day,Day,Day,Day
1,31603,A-31604,2,2016-11-30 16:53:13,2016-11-30 22:53:13,41.73486,-72.66351,41.72755,-72.66619,0.524,...,False,False,False,False,False,False,Night,Night,Day,Day
2,31738,A-31739,2,2016-12-01 06:26:05,2016-12-01 12:26:05,41.10143,-73.43527,41.10643,-73.41528,1.097,...,False,False,False,False,False,False,Night,Night,Day,Day
3,31793,A-31794,2,2016-12-01 08:05:04,2016-12-01 14:05:04,41.28436,-72.94815,41.26920,-72.97343,1.679,...,False,False,False,False,False,False,Day,Day,Day,Day
4,31896,A-31897,2,2016-12-01 11:54:46,2016-12-01 17:54:46,41.15443,-73.24525,41.16724,-73.22949,1.206,...,False,False,False,False,False,False,Day,Day,Day,Day


In [3]:
df.replace({False : int(0), True : int(1)}, inplace=True)

Lots of the features are boolean, so we just switched them to integers. 

In [4]:
df.replace({'Fair':int(0),'Clear':int(0), 
            'Partly Cloudy':int(1),'Scattered Clouds':int(1),'Haze':int(1),'Patches of Fog':int(1),
            'Cloudy':int(2), 'Mostly Cloudy':int(2), 'Overcast':int(2), 'Fair / Windy':int(2),'Mist':int(2),'Smoke':int(2),'Haze / Windy':int(2),
            'Light Rain':int(3),'Cloudy / Windy':int(3),'Fog':int(3), 'Light Drizzle':int(3), 'Thunder in the Vicinity':int(3),'Drizzle':int(3),'Light Freezing Fog':int(3),
            'Light Rain with Thunder':int(3),'Partly Cloudy / Windy':int(3),'Mostly Cloudy / Windy':int(3),'Fog / Windy':int(3),'Heavy Drizzle':int(3),'N/A Precipitation':int(3),
            'Rain':int(4),'Light Snow':int(4),'Heavy Rain':int(4),'Light Rain / Windy':int(4),'Snow':int(4),'Wintry Mix':int(4),'T-Storm':int(4),
            'Heavy T-Storm':int(4), 'Heavy Snow':int(4), 'Snow / Windy': int(4),'Heavy Rain / Windy':int(4),'Rain / Windy':int(4),'Thunder':int(4),
            'Light Freezing Rain':int(4),'Heavy T-Storm / Windy':int(4),'Light Sleet':int(4),'Ice Pellets':int(4),'Heavy Thunderstorms and Rain':int(4),
            'Light Thunderstorms and Rain':int(4), 'Thunderstorms and Rain':int(4),'Heavy Thunderstorms and Snow':int(4),'Light Ice Pellets':int(4),
            'Light Freezing Drizzle':int(4),'Heavy Snow / Windy':int(4),'T-Storm / Windy':int(4),'Thunderstorm':int(4),'Light Snow / Windy':int(4)}, inplace=True)
df['Weather_Condition'].value_counts()

0.0    13463
2.0     9138
3.0     2822
1.0     2691
4.0     1523
Name: Weather_Condition, dtype: int64

We've classified the weather condition features. These are somewhat of up to interpretation, but have been attempeted to be converted from strings to integers in a conservative and consistent manner.

In [5]:
df['Sunrise_Sunset']=df['Sunrise_Sunset'].replace('Day',int(0))
df['Sunrise_Sunset']=df['Sunrise_Sunset'].replace('Night',int(1))
df.rename(columns={'Sunrise_Sunset':'Night'}, inplace=True)
df.iloc[:,44].value_counts()


0.0    18624
1.0    11125
Name: Night, dtype: int64

The 'Sunrise_Sunset' feature actually just tells us if something is night or day. These have been switched to integers.

In [6]:
df['Precipitation(in)']=df['Precipitation(in)']*10
df.rename(columns={'Precipitation(in)':'Precipitation(tenth_in)'}, inplace=True)


for i in df.columns:
    if df[i].dtype == float:
        df[i] = df[i].round().astype('Int64')

Here we switched the 'Precipitation' feature to be measured in tenths of an inch. This is because most of these values in inches are floats in a range of 0-2 inches, so when they are rounded and changed to integers the data is simplified to  values of either 0,1, or 2. By changing these values to tenths, we retain more accuracy in our data.

The loop is just changing every float value in each column to an integer.

In [7]:
rain = df.copy(deep=True)
print(rain['Precipitation(tenth_in)'].mean())
rain['Precipitation(tenth_in)']=df['Precipitation(tenth_in)'].replace(int(0),np.nan)

rain = rain.dropna(subset=['Precipitation(tenth_in)'])


rain['Precipitation(tenth_in)'].mean()



0.07941544885177453


1.9077231695085255

In [8]:
df['Precipitation(tenth_in)'] = df['Precipitation(tenth_in)'].fillna(value=1000)
df.loc[(df['Weather_Condition']==int(4)) & (df['Precipitation(tenth_in)']==1000)  ,'Precipitation(tenth_in)'] = int(2)
df.loc[(df['Weather_Condition']!=int(4)) & (df['Precipitation(tenth_in)']==1000)  ,'Precipitation(tenth_in)'] = int(0)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29762 entries, 0 to 29761
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               29762 non-null  int64 
 1   ID                       29762 non-null  object
 2   Severity                 29762 non-null  int64 
 3   Start_Time               29762 non-null  object
 4   End_Time                 29762 non-null  object
 5   Start_Lat                29762 non-null  Int64 
 6   Start_Lng                29762 non-null  Int64 
 7   End_Lat                  29762 non-null  Int64 
 8   End_Lng                  29762 non-null  Int64 
 9   Distance(mi)             29762 non-null  Int64 
 10  Description              29762 non-null  object
 11  Number                   3630 non-null   Int64 
 12  Street                   29762 non-null  object
 13  Side                     29762 non-null  object
 14  City                     29762 non-nul

In [10]:
#data=np.genfromtxt('CT_Accidents.csv',delimiter=',',skip_header=1)


## PCA Analysis

For PCA analysis we constructed a dataframe with only our relevant numerical values. In doing so, we were hoping to find insight into relationships between the various features and potentially which features could be redundant for insufficient independence.

In [11]:
features = df.columns[[2,22,25,26,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44]]

data = df[features].dropna()
data[features].to_csv(r"C:\Users\kylep\Desktop\MATH_3094\Project\data.csv")

In [12]:
data.head()

,Severity,Temperature(F),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(tenth_in),Weather_Condition,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Night
0,2,48,30,3,6,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,48,30,3,5,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,49,30,10,8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2,49,30,10,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,57,30,10,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
# data_features = ['Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(tenth_in)','Weather_Condition',
# 'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Night']
# x = data.loc[:,data_features].values
# y = data.loc[:,['Severity']].values
#rdata = data

In [14]:
data = data.to_numpy(dtype=int)

In [15]:
data_centered = data - np.mean(data,axis=0)
np.mean(data_centered,axis=0)

array([ 1.11255119e-16,  3.62373815e-15, -9.97322671e-16,  4.80781049e-16,
       -4.76807652e-17, -2.01153228e-17, -6.15876550e-17,  0.00000000e+00,
       -3.72505978e-18,  1.08647577e-19, -1.73836123e-18, -9.31264944e-20,
       -9.31264944e-19,  0.00000000e+00,  5.43237884e-20, -7.45011956e-19,
        0.00000000e+00, -2.17295154e-18,  0.00000000e+00, -5.96009564e-17])

In [16]:
data_centered = StandardScaler().fit_transform(data_centered)
#data_centered = data/np.std(data,axis=0) is another method that did not work due to the small values of the centered data caused by boolean values.

In [17]:
secret_label = data[:,0]

In [18]:
np.argwhere(np.isnan(data).any(axis=1))

array([], shape=(0, 1), dtype=int64)

We converted the dataframe to numpy arrays in order to center and stabalize the data. We tagged severity as the feature to color the scatterplot by since this would be the metric we targeted in Logistic Regression and Naive Bayes analysis.

In [19]:
P=PCA(n_components=2)
PC = P.fit_transform(data_centered)
PC.shape

(28612, 2)

In [20]:
# from bokeh.palettes import Dark2_5 as Dark2_5
# import itertools
# def color_gen():
#     yield from itertools.cycle(Dark2_5)
# colors = [color for color in color_gen()]

In [21]:
colors=['yellow','red','green','orange','red']

#colors_ = [random.choice(colors) for i in range(50)]
color_list = [colors[int(secret_label[i])] for i in range(28612)]

scatter_plot=figure(title='sklearn version')
scatter_plot.scatter(x=PC[:,0],y=PC[:,1],color=color_list)
show(scatter_plot)

Red represents the most severe accidents. While the data grouped into several clusters, different accident types were distributed to all. We were not able to exactly pinpoint what these clusters represent, but the higher clusters seem to be have a higher correlation with the presence of no outlet signs, speed bumps, and nearby emergency stations. The lower clusters seem to have a higher correlation with the status traffif_calming which is mostly taken on high speed or heavy traffic roads. This leads us to hypothesize they could be related to the type of roadway, where there are more accidents overall in smaller more residential areas but the percentage of those being severe is less.

In [22]:

#source=ColumnDataSource({'Severity':data[:,0],'Weather_Condition':data[:,6],'Night':data[:,-1],'Stop':data[:,-5],'No_Exit':data[:,-9]})
names = ['Severity','Temperature(F)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(tenth_in)','Weather_Condition',
'Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Night']
for i in range(20):
    scatter_plot.line(x=[0,1*PC[i,0]],y=[0,1*PC[i,1]], color=Category20[20][i],line_width=3,legend_label=names[i])
scatter_plot.title.text = 'Plot of First Two Principal Components with Feature Loadings'
#scatter_plot.add_tools(HoverTool(tooltips=[("Severity",'$Severity'),('Weather_Condition','@Weather_Conditon'),('Night','@Night'),('Stop','@Stop'),('No_Exit','@No_Exit')]))

show(scatter_plot)

A surprising relation we noticed was that weather condiditon seemed to have little affect on severity. And was usually negatively correlated. We expected this to have a more positive relationship, but the higher proportion of overall accidents occurring during normal conditions is the likely cause of this. This was shown to be the case again in our logistic regression model. There also seemed to be a positive correlation with high severity if a crash occurred at night.

In [23]:
# colors=['red','green','blue','orange','black']
# colors = [random.choice(colour) for i in range(50)]

# color_list = [colors[int(secret_label[i])] for i in range(20000)]
# scatter_plot = figure(title='Plot of First Two Principal Components with secret labels',x_range=(-3,3),y_range=(-3,3))
# scatter_plot.scatter(x=PC2[:,0],y=PC2[:,1],color=color_list)
# show(scatter_plot)

In [24]:
P.components_


array([[-1.26421756e-02, -1.10766678e-01, -2.11160750e-01,
        -6.02792582e-01,  1.39099927e-01,  4.26886100e-01,
         6.12846493e-01, -1.35525272e-20, -2.90663907e-02,
        -5.25482927e-03,  2.22037949e-03, -1.14665038e-02,
        -1.26329660e-02, -0.00000000e+00, -2.20453791e-03,
        -1.94669055e-02, -0.00000000e+00, -2.37154442e-02,
        -0.00000000e+00,  1.86303430e-02],
       [ 1.62570438e-01, -2.92921728e-01,  8.58417822e-02,
        -7.94693937e-02, -2.16716789e-01, -5.20103100e-02,
         3.47967724e-02, -1.73472348e-18,  4.99995798e-01,
         4.04481680e-02, -5.87760723e-02,  7.58629971e-02,
         4.27695584e-01,  0.00000000e+00,  3.29945132e-01,
         2.54565021e-01,  0.00000000e+00,  3.36504905e-01,
         0.00000000e+00,  3.04193338e-01]])

In [25]:
D = np.dot(data_centered.transpose(),data_centered)/200
def r(i,j):
    return D[i,j]/np.sqrt(D[i,i]*D[j,j])
for i in range(15):
    print(i, r(0,i))

0 1.0
1 -0.029198835098650434
2 0.0051470658121838975
3 0.01719267353477376
4 0.03856655919746411
5 -0.012339482510290161
6 0.00467422264169185
7 nan
8 0.056462944551480536
9 0.013733583097926034
10 -0.041737441135766114
11 -0.007674355200336315
12 -0.004178242439864183
13 nan
14 0.010658883844237218


/var/folders/h1/6ljt5t1s68vfzlvj6d2plvj00000gn/T/ipykernel_9501/3451720901.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return D[i,j]/np.sqrt(D[i,i]*D[j,j])


In [26]:
L, P = np.linalg.eigh(D)
L = L[::-1]

In [27]:
eigenvalue_plot = figure(title='Eigenvalues of Covariance Matrix')
eigenvalue_plot.scatter(x=range(L.shape[0]),y=L,size=8)
eigenvalue_plot.line(x=range(L.shape[0]),y=L,color='cyan')
show(eigenvalue_plot)

SerializationError: can't serialize <class 'range'>

## Logistic Regression

For logistic regression, we selected all the numerical columns in the dataset, including those that
we converted previously. This included weather data at the time of the accident and details about 
the area (e.g., whether the accident occured near a railway junction).

Here I have reformated the 'Severity' feature that can act as our target. While we can use the original 4 classes for Linear Regression, restricting it to 2 will let us do our Binary Logistic Regression. 

In the original format, there are only 4 accidents with 'Severity'=1, so switching the classes to just two really shouldnt skew the data much at all. 

In [28]:
df['Severity'].value_counts()

2    24762
4     2896
3     2100
1        4
Name: Severity, dtype: int64

In [29]:
df['Severity']=df['Severity'].replace(int(1),int(0))
df['Severity']=df['Severity'].replace(int(2),int(0))
df['Severity']=df['Severity'].replace(int(3),int(1))
df['Severity']=df['Severity'].replace(int(4),int(1))

df['Severity'].value_counts()

0    24766
1     4996
Name: Severity, dtype: int64

There's a good amount of nan values in each of our numerical feature columns. Here I just dropped every row with them, but we can assess which variables may be better to replace nan values with the average of that feature.

In [30]:
df = df.dropna(subset=['Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Weather_Condition','Night'])
print(df.isnull().values.any())

True


We split the data into testing and training, and trained the model with max iterations of 10,000.

In [31]:
features = df.columns[[22,24,25,26,28,29,30,32,33,34,35,36,37,38,39,40,41,42,43,44]]
x_train, x_test, y_train, y_test = train_test_split(df[features].to_numpy(), df['Severity'].to_numpy())
lr = LogisticRegression(max_iter=10000)
lr.fit(x_train,y_train)

LogisticRegression(max_iter=10000)

After training the model, we got a score of about 80%

In [32]:
pred = lr.predict(x_test)
lr.score(x_test, y_test)

0.8422527318576633

We can plot the coefficients to identify features with significant relationships.

In [33]:
# plt.figure(figsize=(40, 30))
# plt.bar(df[features].columns, L.coef_.ravel())
# plt.show()

# source = ColumnDataSource(data=dict(features=df[features].columns, coeffs=L.coef_.ravel(), color=Spectral16))

p = figure(x_range=df[features].columns.to_list(), width=2000, height=750)
p.vbar(x=df[features].columns, top=lr.coef_.ravel(), width=0.5)
p.xgrid.grid_line_color = None
# p.legend.orientation = "horizontal"
# p.legend.location = "top_center"

show(p)

<!-- Here we list out the features with an abs(coeff) of over 0.4. -->
Here are some features with significant relationships.

In [36]:
coeffs = pd.DataFrame({
    "Feature": df[features].columns,
    "Coeff": lr.coef_.ravel(),
})
coeffs[abs(coeffs["Coeff"]) > 0.4]

,Feature,Coeff
8,Crossing,0.463551
9,Give_Way,0.572302
11,No_Exit,-1.108106
12,Railway,-0.715534
14,Station,-0.747742
15,Stop,1.507724
17,Traffic_Signal,1.361884


Confusion matrix

In [37]:
cm = confusion_matrix(y_test, pred)
cm

array([[5992,   18],
       [1108,   20]])

Statistics to evaluate the model

In [38]:
tn, fp, fn, tp = cm.ravel()
pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1"],
    "Value": [(tp+tn)/(tp+fp+fn+tn), tp/(tp+fp), tp/(tp+fn), (2*tp/(tp+fp))*(tp/(tp+fn)) / ((tp/(tp+fp))+(tp/(tp+fn)))]
})

,Metric,Value
0,Accuracy,0.842253
1,Precision,0.526316
2,Recall,0.017730
3,F1,0.034305


## Naive Bayes

For this section, we used the description column of the dataset and ran sentiment analysis of it
against the severity label. We wanted to see if there specific words that would indicate a higher
severity.

Like with logistic regression, we first split the data into training and testing data.

In [39]:
train_descriptions, test_descriptions, train_labels, test_labels = train_test_split(df["Description"].to_numpy(), df["Severity"].to_numpy(), test_size=0.25, random_state=11)

We created a `CountVectorizer` to split the description into each word and count up the number
of each word. We used this fectorizer to fit our training data and ran a Bernoulli Naive Bayes
model.

In [40]:
vectorizer = CountVectorizer(max_features=100, binary=True, stop_words="english")
x_train = vectorizer.fit_transform(train_descriptions)
y_train = np.array(train_labels)
bernoulli = BernoulliNB().fit(x_train, y_train)

Here we format the test data and transform it with the vectorizer.

In [41]:
x_test = vectorizer.transform(test_descriptions)
y_test = np.array(test_labels)

Now we run the model with the test data.

In [42]:
predictions = bernoulli.predict(x_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

We get an accuracy score of about 80%.

In [43]:
bernoulli.score(x_test, y_test)

0.8096105351639115

To evaluate the model, we show the confusion matrix and metrics.

In [44]:
cm = confusion_matrix(y_test, predictions)
cm

array([[4775, 1196],
       [ 163, 1004]])

In [45]:
tn, fp, fn, tp = cm.ravel()
pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [ (tp+tn)/(tp+fp+fn+tn), tp/(tp+fp), tp/(tp+fn),  (2*tp/(tp+fp))*(tp/(tp+fn)) / ((tp/(tp+fp))+(tp/(tp+fn)))]
})

,Metric,Value
0,Accuracy,0.809611
1,Precision,0.456364
2,Recall,0.860326
3,F1 Score,0.596377


We list out the words that correspond to a high and low severity, respectively.

In [46]:
train_matrix = vectorizer.fit_transform(train_descriptions).toarray()
keywords = vectorizer.get_feature_names_out()
train_y = y_train
freq_plus = (train_y.transpose()@train_matrix)
Nplus = train_y.transpose()@train_y
Pplus = freq_plus/Nplus
freq_minus = ((1-train_y).transpose()@train_matrix)
Nminus = ((1-train_y).transpose()@(1-train_y))
Pminus = freq_minus/Nminus
N = Nminus+Nplus

In [47]:
indices = np.argsort(Pplus)
[keywords[i] for i in indices[::-1]][:20]

['accident',
 'exit',
 'closed',
 'road',
 'ct',
 'rd',
 'st',
 'blocked',
 'ave',
 'lane',
 'vehicle',
 'route',
 'reported',
 'lanes',
 'near',
 'alternate',
 'incident',
 'main',
 'right',
 'left']

In [48]:
indices = np.argsort(Pminus)
[keywords[i] for i in indices[::-1]][:20]

['exit',
 'accident',
 'lane',
 'reported',
 'closed',
 'vehicle',
 'exits',
 'ct',
 'traffic',
 '95',
 'pm',
 'crash',
 'right',
 'left',
 '84',
 'near',
 'incident',
 'conndot',
 'blocked',
 'st']

Here is a plot of all the words with respect to their relationship with high and low severity.

In [49]:
source = ColumnDataSource({'+':Pplus,'-':Pminus,'word':keywords})
f=figure()
f.scatter(x='+',y='-',source=source)

f.xaxis.axis_label='P(w|+)'
f.yaxis.axis_label = 'P(w|-)'
f.line(x=[0,.2],y=[0,.2])
f.add_tools(HoverTool(tooltips=[("word","@word")]))
show(f)